<a href="https://colab.research.google.com/github/candle16/statcan-crime-analysis/blob/main/Crime_Analysis_Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:

# Configure Git with your info
!git config --global user.name "candle16"
!git config --global user.email "kateskipton@gmail.com"

# Store credentials so you don't have to enter token every time
!git config --global credential.helper store.

In [23]:
# Clone your repository
!git clone https://github.com/candle16/statcan-crime-analysis.git

# Navigate into the folder
%cd statcan-crime-analysis

# Check that you're in the right place
!pwd

Cloning into 'statcan-crime-analysis'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.
/content/statcan-crime-analysis/statcan-crime-analysis/statcan-crime-analysis
/content/statcan-crime-analysis/statcan-crime-analysis/statcan-crime-analysis


In [24]:
!mkdir -p data
!mkdir -p outputs
!mkdir -p scripts

In [25]:
print('Hello world')

Hello world


In [26]:
# CELL 1: Install Dependencies
!pip install stats-can openpyxl

In [27]:
%%writefile scripts/analysis.py

# CELL 2: Import Libraries
import pandas as pd
import stats_can as sc
import logging
from pathlib import Path
from IPython.display import display

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(message)s'
)
logger = logging.getLogger(__name__)

Writing scripts/analysis.py


In [ ]:
# ============================================================================
# STATISTICS CANADA CRIME DATA DOWNLOAD
# ============================================================================
# This cell downloads multiple crime-related tables from Statistics Canada
# using the stats_can library. All data is fetched for Canada-level geography
# across all available years.
#
# Tables being downloaded:
# - 35-10-0001-01: Incident-based crime statistics, by detailed violations
# - 35-10-0177-01: Incident-based crime statistics, by detailed violations (Revised)
# - 35-10-0062-01: Crime severity index and weighted clearance rates
# - 35-10-0026-01: Police-reported crime for selected offences
#
# The data will be saved to the 'data/' directory for subsequent analysis.
# ============================================================================

import pandas as pd
import stats_can as sc
from pathlib import Path
from datetime import datetime
import sys
import time

# Create data directory if it doesn't exist
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

# Define the Statistics Canada tables to download
STATCAN_TABLES = {
    '35-10-0001-01': 'incident_based_crime_detailed',
    '35-10-0177-01': 'incident_based_crime_revised',
    '35-10-0062-01': 'crime_severity_index',
    '35-10-0026-01': 'police_reported_crime_selected'
}

print("=" * 80)
print("STATISTICS CANADA CRIME DATA DOWNLOAD")
print("=" * 80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Number of tables to download: {len(STATCAN_TABLES)}")
print("=" * 80)
print()

# Dictionary to store downloaded dataframes
crime_data = {}
download_summary = []

# Download each table
for table_id, table_name in STATCAN_TABLES.items():
    print(f"📊 Processing Table: {table_id}")
    print(f"   Description: {table_name}")
    print(f"   {'─' * 70}")
    
    try:
        # Record start time for this table
        start_time = time.time()
        
        # Download the table
        # The stats_can library automatically fetches all available data
        print(f"   ⏳ Downloading data from Statistics Canada...")
        df = sc.sc.table_to_df(table_id)
        
        # Calculate download time
        download_time = time.time() - start_time
        
        # Filter for Canada-level geography if GEO column exists
        if 'GEO' in df.columns:
            canada_df = df[df['GEO'].str.contains('Canada', case=False, na=False)].copy()
            print(f"   ✓ Filtered for Canada geography")
            print(f"   ✓ Records (Canada): {len(canada_df):,}")
        else:
            canada_df = df.copy()
            print(f"   ℹ No GEO column found - using all records")
        
        # Store the dataframe
        crime_data[table_name] = canada_df
        
        # Save to CSV
        output_file = data_dir / f"{table_name}_{table_id.replace('-', '_')}.csv"
        canada_df.to_csv(output_file, index=False)
        
        # Get year range if REF_DATE column exists
        year_range = "N/A"
        if 'REF_DATE' in canada_df.columns:
            years = canada_df['REF_DATE'].dropna().astype(str)
            if len(years) > 0:
                # Extract years from various date formats
                year_values = pd.to_datetime(years, errors='coerce').dt.year.dropna()
                if len(year_values) > 0:
                    year_range = f"{year_values.min()} - {year_values.max()}"
        
        # Display summary
        print(f"   ✓ Download complete!")
        print(f"   ✓ Total records: {len(df):,}")
        print(f"   ✓ Columns: {len(canada_df.columns)}")
        print(f"   ✓ Year range: {year_range}")
        print(f"   ✓ Download time: {download_time:.2f} seconds")
        print(f"   ✓ Saved to: {output_file}")
        
        # Store summary
        download_summary.append({
            'Table ID': table_id,
            'Table Name': table_name,
            'Total Records': len(df),
            'Canada Records': len(canada_df),
            'Columns': len(canada_df.columns),
            'Year Range': year_range,
            'Download Time (s)': f"{download_time:.2f}",
            'File Size (MB)': f"{output_file.stat().st_size / (1024*1024):.2f}",
            'Status': '✓ Success'
        })
        
        print(f"   {'─' * 70}")
        print()
        
    except Exception as e:
        print(f"   ✗ ERROR downloading {table_id}: {str(e)}")
        print(f"   {'─' * 70}")
        print()
        
        # Store error in summary
        download_summary.append({
            'Table ID': table_id,
            'Table Name': table_name,
            'Total Records': 0,
            'Canada Records': 0,
            'Columns': 0,
            'Year Range': 'N/A',
            'Download Time (s)': '0',
            'File Size (MB)': '0',
            'Status': f'✗ Error: {str(e)[:50]}'
        })
        
        continue

# Display download summary
print("=" * 80)
print("DOWNLOAD SUMMARY")
print("=" * 80)

summary_df = pd.DataFrame(download_summary)
print(summary_df.to_string(index=False))

print()
print("=" * 80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Successfully downloaded: {len([s for s in download_summary if 'Success' in s['Status']])} / {len(STATCAN_TABLES)} tables")
print(f"Data saved to: {data_dir.absolute()}")
print("=" * 80)

# Display a preview of the first dataset
if crime_data:
    first_table = list(crime_data.keys())[0]
    print()
    print(f"📋 Preview of {first_table}:")
    print("=" * 80)
    print(crime_data[first_table].head(10))
    print()
    print(f"Shape: {crime_data[first_table].shape}")
    print(f"Columns: {list(crime_data[first_table].columns)}")